In [1]:
import datetime
import os
import random
import readline
from datetime import date

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cftime
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
import netCDF4
import numpy
import pandas
import scipy
import seaborn
import xarray as xr
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
from matplotlib import colors, font_manager, pyplot
from matplotlib.cm import ScalarMappable
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
from scipy import fft, linalg, stats
from scipy.stats import poisson, ttest_ind
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import normalize
from statsmodels.tsa.ar_model import AutoReg

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/RCP_Files_V4/'

In [3]:
# Open File
def Create_DF(File):
    Data = open(File, 'r')
    Rows = []
#
# Organize Data
    for Line in Data:
        Rows.append(Line.strip())
    Storm_Code = []
    Storm_List_Orig = []
    for i in range(len(Rows)):
        if Rows[i][0:5] == 'start':
            Code = Rows[i][41:45]
            Storm_List_Orig.append(Code)
        else:
            Storm_Code.append(Code)
    Array = numpy.zeros((13, len(Rows)-len(Storm_List_Orig)))
    Time = []
    k = -1
    for i in range(len(Rows)):
#        print (i)
        if Rows[i][0:5] == 'start':
            k += 1
        else:
            l = len(Rows[i]) - 100
            Array[0][i-k-1] = float(Rows[i][0:6+l])
            Array[1][i-k-1] = float(Rows[i][8+l:14+l])
            Array[2][i-k-1] = float(Rows[i][17+l:24+l])
            Array[3][i-k-1] = float(Rows[i][26+l:31+l])
            Array[4][i-k-1] = float(Rows[i][34+l:41+l])
            Array[5][i-k-1] = float(Rows[i][44+l:51+l])
            Array[6][i-k-1] = float(Rows[i][54+l:61+l])
            Array[7][i-k-1] = float(Rows[i][64+l:71+l])
            Array[8][i-k-1] = float(Rows[i][74+l:81+l])
            Time.append(datetime.datetime(year=int(Rows[i][84+l:88+l]), month=int(Rows[i][90+l:92+l]), \
            day=int(Rows[i][94+l:96+l]), hour=int(Rows[i][98+l:100+l])))
#
# Create DataFrame to Store Data
    DF_All = pandas.DataFrame({"Orig Code": Storm_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "Dist(m)": Array[4], "Angle": Array[5], "B": Array[6], "VLT": Array[7], "VUT": Array[8], \
    "Orig Time(Z)": Time})
#
# Remove Data Points Missing Phase Space Parameter Values (-999.0)
    DF = DF_All[(DF_All["B"] > -728) & (DF_All["VLT"] > -728) & (DF_All["VUT"] > -728)].reset_index()
    return (DF, Storm_List_Orig)

In [4]:
# Combine The Three Datasets Into One DataFrame of 90 or 93 Years For Each Scenario
def Combine_DF(DF_A, DF_B, DF_C, Orig_List_A, Orig_List_B, Orig_List_C, Model):
# Change Date For Simplicity of Analysis
# Control Assigned to Years in the 20th Century
# Each Year Within the Century are Independent
    if Model == "Control":
        Year_Start = 1900
# RCP4.5 Assigned to Years in the 20th Century
    elif Model == "RCP45":
        Year_Start = 2000
# RCP8.5 Assigned to Years in the 20th Century
    elif Model == "RCP85":
        Year_Start = 2100
# Create Empty Lists
    Origi_List_A = []
    Origi_List_B = []
    Origi_List_C = []
    New_List_A = []
    New_List_B = []
    New_List_C = []
    New_Time = []
    New_Code = []
    ABC = []
    Orig_Combine = []
    New_Combine = []
# Dataset A
    Year = Year_Start
    Orig_Year = DF_A["Orig Time(Z)"][0].year
    Year_Diff_A = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_A)):
        try:
            Code, Year, Count = Update_Code(DF_A, Orig_List_A[k], Year_Diff_A, Year, Count)
            Origi_List_A.append(Orig_List_A[k])
            New_List_A.append(Code)
        except:
            Kinen = 728
# Dataset B
    Year += 1
    Orig_Year = DF_B["Orig Time(Z)"][0].year
    Year_Diff_B = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_B)):
        try:
            Code, Year, Count = Update_Code(DF_B, Orig_List_B[k], Year_Diff_B, Year, Count)
            Origi_List_B.append(Orig_List_B[k])
            New_List_B.append(Code)
        except:
            Kinen = 728
# Dataset C
    Year += 1
    Orig_Year = DF_C["Orig Time(Z)"][0].year
    Year_Diff_C = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_C)):
        try:
            Code, Year, Count = Update_Code(DF_C, Orig_List_C[k], Year_Diff_C, Year, Count)
            Origi_List_C.append(Orig_List_C[k])
            New_List_C.append(Code)
        except:
            Kinen = 728
#
# Combine Into New Time and Codes List
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_A, Origi_List_A, New_List_A, Year_Diff_A, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "A")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_B, Origi_List_B, New_List_B, Year_Diff_B, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "B")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_C, Origi_List_C, New_List_C, Year_Diff_C, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "C")
#
# Concatenate Other Variables
    Vars = ["Lon", "Lat", "SLP(hPa)", "Winds(m/s)", "Dist(m)", "Angle", "B", "VLT", "VUT"]
    Total_Length = len(New_Code)
    Array = numpy.zeros((9,Total_Length))
    for n in range(len(Vars)):
        List = []
        for k in range(len(DF_A)):
            List.append(DF_A[Vars[n]][k])
        for k in range(len(DF_B)):
            List.append(DF_B[Vars[n]][k])
        for k in range(len(DF_C)):
            List.append(DF_C[Vars[n]][k])
        Array[n] = List
#
# Create DataFrame to Store Data
    DF_Combine = pandas.DataFrame({"Code": New_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "B": Array[6], "VLT": Array[7], "VUT": Array[8], "Time(Z)": New_Time})
    Codes_DF = pandas.DataFrame({"ABC": ABC, "Orig Code": Orig_Combine, "New Code": New_Combine})
    Year_Diffs = numpy.array([Year_Diff_A, Year_Diff_B, Year_Diff_C])
    return (DF_Combine, Codes_DF, Year_Diffs)

In [5]:
# Update Storm Code to Contain Year and Storm Number
def Update_Code(DF, Orig_Code, Year_Diff_A, Year, Count):
    DF_Storm = DF[DF["Orig Code"] == Orig_Code].reset_index()
    Orig_Year = DF_Storm["Orig Time(Z)"][0].year
    Time_Year = Orig_Year + Year_Diff_A
    if Time_Year == Year:
        Count += 1
    else:
        Year += 1
        Count = 1
    if Count < 10:
        Code = "TC"+str(Year)+"0"+str(Count)
    else:
        Code = "TC"+str(Year)+str(Count)
    return (Code, Year, Count)

In [6]:
# Combine Lists
def Combine_Lists(DF, Orig_List, New_List, Year_Diff, New_Code, New_Time, ABC, Orig_Combine, New_Combine, Y):
    for i in range(len(DF)):
        for l in range(len(Orig_List)):
# Update Storm Code
            if DF["Orig Code"][i] == Orig_List[l]:
                New_Code.append(New_List[l])
# Update Year
                Orig_Time = DF["Orig Time(Z)"][i]
                New_Time.append(Update_Year(Orig_Time, Year_Diff))
    for m in range(len(Orig_List)):
        ABC.append(Y)
        Orig_Combine.append(Orig_List[m])
        New_Combine.append(New_List[m])
    return (New_Code, New_Time, ABC, Orig_Combine, New_Combine)

In [7]:
# Change Year of Data
def Update_Year(Orig_Time, Year_Diff):
    Year_Update = Orig_Time.year + Year_Diff
# Febuary 29 Problems
    if Orig_Time.month == 2 and Orig_Time.day == 29:
        New_Time = Orig_Time.replace(year=Year_Update, day=28)
    else:
        New_Time = Orig_Time.replace(year=Year_Update)
    return (New_Time)

In [8]:
# Open Name List File
def Open_Name_List(File):
    Name_File = pandas.read_csv(File)
    Headings = list(Name_File.columns)
    Name_List = []
    for i in range(len(Headings)):
        for j in range(len(Name_File)):
            Name_List.append(Name_File[Headings[i]][j])
    return (Name_List)

In [9]:
# Assign Names to Each Cyclone Code
def Assign_Name(Name_List, Codes, DF):
    Storm_Names = []
    for i in range(len(Codes)):
        Name = Name_List[int(i%len(Name_List))]
        Storm_Names.append(Name)
    Codes["Name"] = Storm_Names
    DF_Names = []
    for k in range(len(DF)):
        Code = DF["Code"][k]
        Name = list(Codes[Codes["New Code"] == Code]["Name"])[0]
        DF_Names.append(Name)
    DF.insert(1, "Name", DF_Names)
    return (Codes, DF)

In [10]:
# Function to Find Distance Between Two Points
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [11]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [12]:
# Open Name List
Name_List = Open_Name_List('Storm_Name_List.csv')

In [13]:
# Apply Functions to Open Datasets
Control_A_DF_Orig, Control_A_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.control.002_avg_V4')
Control_B_DF_Orig, Control_B_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.control.003_avg_V4')
Control_C_DF_Orig, Control_C_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.control.004_avg_V4')

In [14]:
# Process Control DataFrames
Control_DF_Combined, Control_Codes, Control_Year_Diffs = \
Combine_DF(Control_A_DF_Orig, Control_B_DF_Orig, Control_C_DF_Orig, \
Control_A_Storm_List_Orig, Control_B_Storm_List_Orig, Control_C_Storm_List_Orig, "Control")
Control_Codes_Names, Control_DF_Names = Assign_Name(Name_List, Control_Codes, Control_DF_Combined)

In [15]:
# Apply Functions to Open Datasets
RCP45_A_DF_Orig, RCP45_A_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp45.001_avg_V4')
RCP45_B_DF_Orig, RCP45_B_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp45.002_avg_V4')
RCP45_C_DF_Orig, RCP45_C_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp45.003_avg_V4')

In [16]:
# Process RCP4.5 DataFrames
RCP45_DF_Combined, RCP45_Codes, RCP45_Year_Diffs = \
Combine_DF(RCP45_A_DF_Orig, RCP45_B_DF_Orig, RCP45_C_DF_Orig, \
RCP45_A_Storm_List_Orig, RCP45_B_Storm_List_Orig, RCP45_C_Storm_List_Orig, "RCP45")
RCP45_Codes_Names, RCP45_DF_Names = Assign_Name(Name_List, RCP45_Codes, RCP45_DF_Combined)

In [17]:
# Apply Functions to Open Datasets
RCP85_A_DF_Orig, RCP85_A_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp85.001_avg_V4')
RCP85_B_DF_Orig, RCP85_B_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp85.003_avg_V4')
RCP85_C_DF_Orig, RCP85_C_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp85.004_avg_V4')

In [18]:
# Process RCP8.5 DataFrames
RCP85_DF_Combined, RCP85_Codes, RCP85_Year_Diffs = \
Combine_DF(RCP85_A_DF_Orig, RCP85_B_DF_Orig, RCP85_C_DF_Orig, \
RCP85_A_Storm_List_Orig, RCP85_B_Storm_List_Orig, RCP85_C_Storm_List_Orig, "RCP85")
RCP85_Codes_Names, RCP85_DF_Names = Assign_Name(Name_List, RCP85_Codes, RCP85_DF_Combined)

In [19]:
Control_A_DF_Orig

,index,Orig Code,Lon,Lat,SLP(hPa),Winds(m/s),Dist(m),Angle,B,VLT,VUT,Orig Time(Z)
0,0,0024,-82.19,27.99,1015.45,13.9,-999.00,-999.00,4.71,37.04,-36.64,1985-06-19 12:00:00
1,1,0024,-82.26,27.79,1015.45,13.8,23.61,197.20,6.18,-12.97,-23.15,1985-06-19 18:00:00
2,2,0024,-82.33,27.58,1015.45,13.7,23.61,197.23,3.28,-8.22,-18.60,1985-06-20 00:00:00
3,3,0024,-82.40,27.38,1015.45,13.7,23.62,197.27,3.45,-0.38,-18.54,1985-06-20 06:00:00
4,4,0024,-82.46,28.31,1014.22,11.7,103.18,356.87,1.72,8.23,-4.21,1985-06-20 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
9952,10548,1525,-73.07,32.06,978.45,39.7,348.36,23.39,55.03,76.84,-215.91,2014-11-12 00:00:00
9953,10549,1525,-72.43,36.23,974.77,43.7,467.34,6.94,70.07,-26.61,-227.50,2014-11-12 06:00:00
9954,10550,1525,-75.16,38.47,986.59,28.1,346.75,316.71,79.01,-119.69,-217.65,2014-11-12 12:00:00
9955,10551,1525,-77.88,40.72,998.40,12.5,341.66,317.64,89.89,-222.06,-203.50,2014-11-12 18:00:00


In [20]:
# Check For Repeated Entries
def Repeated_Check(DF):
    for i in range(len(DF)):
        Lat = DF["Lat"][i]
        Lon = DF["Lon"][i]
        Time = DF["Time(Z)"][i]
        DF_Repeat = DF[(DF["Lat"] == Lat) & (DF["Lon"] == Lon) & (DF["Time(Z)"] == Time)]
# Deleted Repeated Entries
        if len(DF_Repeat) > 1:
            print (DF["Code"][i], Time)

In [21]:
#DF_Storm = Find_Storm(RCP85_DF_Names, "TC216105")
#DF_Storm

In [22]:
#Repeated_Check(RCP85_DF_Names)

In [23]:
# Create DF With Extratropical Transition Information of Each Storm
def Storm_Info_DF(DF, Codes):
    Path_Types = []
    Start_Times = []
    End_Times = []
    Track_Discons = []
    for i in range(len(Codes)):
        DF_Storm_Orig = DF[DF["Code"] == Codes["New Code"][i]].reset_index()
        DF_Storm, Track_Discons = DF_Fixing(DF_Storm_Orig, Track_Discons)
        Storm_Type, Trans_Type, Start_Time_k, End_Time_k = ET_Function(DF_Storm)
        Start_Time = DF_Storm["Time(Z)"][Start_Time_k]
        if End_Time_k > 0:
            End_Time = DF_Storm["Time(Z)"][End_Time_k]
        else:
            End_Time = numpy.nan
        Start_Times.append(Start_Time)
        End_Times.append(End_Time)
        Path_Types.append(Trans_Type)
        if i == 0:
            DF_Fixed = DF_Storm.copy()
        else:
            DF_Fixed = pandas.concat([DF_Fixed, DF_Storm])
#
# Create Storms DF
    Storms_DF_Orig = pandas.DataFrame({"Code": Codes["New Code"], "Name": Codes["Name"], \
    "Path Type": Path_Types, "Start Time": Start_Times, "End Time": End_Times})
    return (DF_Fixed, Storms_DF_Orig)

In [24]:
# Function For Check For Time Discontinuity
def Time_Discontinuity(DF_Storm):
    Time_Continuous = 0
    for k in range(len(DF_Storm)):
        if k > 0 and Time_Continuous == 0:
#            print (k)
# If Over 12 Hours of Missing Data
            if DF_Storm["Time(Z)"][k] - DF_Storm["Time(Z)"][k-1] > datetime.timedelta(hours=12):
# Divide Array Into Before The Time Discontinuity and After It
                Before = DF_Storm["Time(Z)"][k-1]
                After = DF_Storm["Time(Z)"][k]
                DF_Before = DF_Storm[DF_Storm["Time(Z)"] <= Before].reset_index()
                DF_After = DF_Storm[DF_Storm["Time(Z)"] >= After].reset_index()
# Choose the Part with the Lowest Min SLP to Keep
                if numpy.min(DF_After["SLP(hPa)"]) <= numpy.min(DF_Before["SLP(hPa)"]) and \
                len(DF_After) > len(DF_Before):
                    DF_Storm_Updated = DF_After
                else:
                    DF_Storm_Updated = DF_Before
                Time_Continuous = 1
                DF_Storm_Updated = DF_Storm_Updated.drop("level_0", axis=1)
# Repeat Until No More Time Discontinuity
    if Time_Continuous == 0:
        DF_Storm_Updated = DF_Storm.copy()
    return (DF_Storm_Updated)

In [25]:
# Function For Check For Track Discontinuity
def Track_Discontinuity(DF_Storm):
    Track_Continuous = 0
    Track_Discon = []
    for k in range(len(DF_Storm)):
        if k > 1 and Track_Continuous == 0:
# Calculate Distance Between Each 6 Hour Interval
            Distance_1 = Find_Distance(DF_Storm["Lat"][k], DF_Storm["Lat"][k-1], \
            DF_Storm["Lon"][k], DF_Storm["Lon"][k-1])
            Distance_2 = Find_Distance(DF_Storm["Lat"][k-1], DF_Storm["Lat"][k-2], \
            DF_Storm["Lon"][k-1], DF_Storm["Lon"][k-2])
# If Distance >= 800km and Distance Over 5 Times Further Than Previous Time Interval
            if Distance_1 >= 800 and Distance_1 >= Distance_2 * 5:
                Track_Discon = [DF_Storm["Code"][0], DF_Storm["Time(Z)"][k]]
# Divide Array Into Before The Track Discontinuity and After It
                Before = DF_Storm["Time(Z)"][k-1]
                After = DF_Storm["Time(Z)"][k]
                DF_Before = DF_Storm[DF_Storm["Time(Z)"] <= Before].reset_index()
                DF_After = DF_Storm[DF_Storm["Time(Z)"] >= After].reset_index()
# Choose the Part with the Lowest Min SLP to Keep
                if numpy.min(DF_After["SLP(hPa)"]) <= numpy.min(DF_Before["SLP(hPa)"]) and \
                len(DF_After) > len(DF_Before):
                    DF_Storm_Updated = DF_After
                else:
                    DF_Storm_Updated = DF_Before
                Track_Continuous = 1
                DF_Storm_Updated = DF_Storm_Updated.drop("level_0", axis=1)
# Repeat Until No More Track Discontinuity
    if Track_Continuous == 0:
        DF_Storm_Updated = DF_Storm.copy()
    return (DF_Storm_Updated, Track_Discon)

In [26]:
# Fix Storms That Have Time Discontinuity or Track Discontinuity
def DF_Fixing(DF_Storm_Orig, Track_Discons):
    Bottom_Right = False
    Top_Right = False
    Bottom_Left = False
    Top_Left = False
    Bottom_Right_k = []
    Top_Right_k = []
    Bottom_Left_k = []
    ET_Complete = False
#
# Check For Time Discontinuity
    DF_Storm = Time_Discontinuity(DF_Storm_Orig)
# Repeat to Check Again
    DF_Storm = Time_Discontinuity(DF_Storm)
    DF_Storm = Time_Discontinuity(DF_Storm)
#
# Check If Track Continous
    DF_Storm, Track_Discon = Track_Discontinuity(DF_Storm)
    if len(Track_Discon) > 0:
        Track_Discons.append(Track_Discon)
# Repeat to Check Again
    DF_Storm, Track_Discon = Track_Discontinuity(DF_Storm)
    if len(Track_Discon) > 0:
        Track_Discons.append(Track_Discon)
    DF_Storm, Track_Discon = Track_Discontinuity(DF_Storm)
    if len(Track_Discon) > 0:
        Track_Discons.append(Track_Discon)
    return (DF_Storm, Track_Discons)

In [27]:
# Function to Find Whether a Storm Has Completed ET, and Start and End Time of ET
def ET_Function(DF_Storm):
    Tropical_Begin = False
    ET_Begin = False
    ET_Complete = False
    Bottom_Right_k = []
    Top_Right_k = []
    Bottom_Left_k = []
    Start_Time_k = 0
    End_Time_k = 0
    Storm_Type = 0
    Trans_Type = 0
    for k in range(len(DF_Storm)):
        if ET_Complete == False:
            if Tropical_Begin == False:
# Storm Must First Be Tropical
                if DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
                    Tropical_Begin = True
                    Bottom_Right_k.append(k)
            if Tropical_Begin == True:
# Storm Must Be Tropical For At Least 48 Hours
                if len(Bottom_Right_k) <= 8:
                    if DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
                        Bottom_Right_k.append(k)
# Enter Next Stage If Storm Remains Tropical For At Least 48 Hours
                else:
                    if ET_Begin == False:
# If Storm Entes Top Right Quadrant = ET Begin
                        if DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] >= 0:
                            ET_Begin = True
                            Top_Right_k.append(k)
# If Storm Entes Bottom Left Quadrant = ET Begin
                        elif DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] < 0:
                            ET_Begin = True
                            Bottom_Left_k.append(k)
# If Storm Directly Enters Top Left Quadrant = Completed ET
                        elif DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] < 0:
                            ET_Complete = True
                            Start_Time_k = k
                            End_Time_k = k
                            Trans_Type = 3
                    elif ET_Begin == True:
                        if len(Top_Right_k) > 0:
                            if DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] >= 0:
                                Top_Right_k.append(k)
# Storm Needs to Stay Within Top Right Quadrant For At Least 24 Hours
                                if len(Top_Right_k) == 4:
                                    Start_Time_k = Top_Right_k[0]
# If Storm Enters Bottom Left Quadrant Within 24 Hours After Leaving Top Right Quadrant = Completed ET
                            elif DF_Storm["VLT"][k] < 0:
                                ET_Complete = True
                                Start_Time_k = Top_Right_k[0]
                                End_Time_k = k
                                Trans_Type = 1
                            elif DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
# Reset the Counting if Storm Returns to Bottom Right Quadrant For Over 24 Hours
                                if k-4 > Top_Right_k[len(Top_Right_k)-1]:
                                    Top_Right_k = []
                        elif len(Bottom_Left_k) > 0:
                            if DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] < 0:
                                Bottom_Left_k.append(k)
# Storm Needs to Stay Within Bottom Left Quadrant For At Least 24 Hours
                                if len(Bottom_Left_k) == 4:
                                    Start_Time_k = Bottom_Left_k[0]
# If Storm Enters Top Right Quadrant Within 24 Hours After Leaving Top Right Quadrant = Completed ET
                            elif DF_Storm["B"][k] > 15:
                                ET_Complete = True
                                Start_Time_k = Bottom_Left_k[0]
                                End_Time_k = k
                                Trans_Type = 2
                            elif DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
# Reset the Counting if Storm Returns to Bottom Right Quadrant For Over 24 Hours
                                if k-4 > Bottom_Left_k[len(Bottom_Left_k)-1]:
                                    Bottom_Left_k = []
# Repeat Process Again After Storm Returns to Bottom Right Quadrant Then Leaves Again
                        elif len(Top_Right_k) == 0 and len(Bottom_Left_k) == 0:
                            if DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] >= 0:
                                Top_Right_k.append(k)
                            elif DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] < 0:
                                Bottom_Left_k.append(k)
                            elif DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] < 0:
                                ET_Complete = True
                                Start_Time_k = k
                                End_Time_k = k
                                Trans_Type = 3
#
# If ET Never Begun:
    if ET_Complete == False:
        if ET_Begin == False:
            Trans_Type = -1
            Start_Time_k = len(DF_Storm)-1
            End_Time_k = -728
            Storm_Type = "No ET Transition"
# If ET Began But Did Not Complete:
        elif ET_Begin == True:
            Trans_Type = 0
            End_Time_k = -728
            Storm_Type = "ET Transition Incomplete"
# If ET Completed:
    else:
        Storm_Type = "ET Transition Complete"
    return (Storm_Type, Trans_Type, Start_Time_k, End_Time_k)

In [28]:
#DF_Storm = Find_Storm(RCP85_DF_Names, "TC216105")
#DF_Storm

In [29]:
# Apply Function For Control DataFrames
Control_DF_Fixed, Control_Storms_DF_Orig = Storm_Info_DF(Control_DF_Names, Control_Codes_Names)

In [30]:
# Apply Function For RCP4.5 DataFrames
RCP45_DF_Fixed, RCP45_Storms_DF_Orig = Storm_Info_DF(RCP45_DF_Names, RCP45_Codes_Names)

In [31]:
# Apply Function For RCP8.5 DataFrames
RCP85_DF_Fixed, RCP85_Storms_DF_Orig = Storm_Info_DF(RCP85_DF_Names, RCP85_Codes_Names)

In [32]:
# Create ET DF From Storms DF With Just Storms That Completed ET Transition
def ET_DF_Filter(Main_DF, Storms_DF_Orig, Codes_DF):
    ET_DF = Storms_DF_Orig[Storms_DF_Orig["Path Type"] > 0].reset_index()
    ET_DF = ET_DF.drop("index", axis=1)
    Storms_DF = Storms_DF_Orig.copy()
#
# Filter For Just Storms That Completed ET Transition
    Code_List = ET_DF["Code"]
    Filtered_DF = Main_DF[Main_DF["Code"].isin(Code_List)].reset_index()
    Filtered_Codes = Codes_DF[Codes_DF["New Code"].isin(Code_List)].reset_index()
    Filtered_DF = Filtered_DF.drop("index", axis=1)
    Filtered_Codes = Filtered_Codes.drop("index", axis=1)
    return (ET_DF, Storms_DF, Filtered_DF, Filtered_Codes)

In [33]:
# Apply Function
Control_ET_Orig, Control_Storms_Final, Control_DF_Filtered, Control_Codes_Final = \
ET_DF_Filter(Control_DF_Fixed, Control_Storms_DF_Orig, Control_Codes_Names)
RCP45_ET_Orig, RCP45_Storms_Final, RCP45_DF_Filtered, RCP45_Codes_Final = \
ET_DF_Filter(RCP45_DF_Fixed, RCP45_Storms_DF_Orig, RCP45_Codes_Names)
RCP85_ET_Orig, RCP85_Storms_Final, RCP85_DF_Filtered, RCP85_Codes_Final = \
ET_DF_Filter(RCP85_DF_Fixed, RCP85_Storms_DF_Orig, RCP85_Codes_Names)

In [34]:
# Add Storm Phase into DataFrame
def Storm_Phase_Info(Main_DF, ET_DF):
    Storm_Phase_List = []
    for i in range(len(ET_DF)):
        DF_Storm = Main_DF[Main_DF["Code"] == ET_DF["Code"][i]].reset_index()
        Start_Time = ET_DF["Start Time"][i]
        End_Time = ET_DF["End Time"][i]
# Find Storm Phase Based on ET Start Time and ET End Time
        Storm_Phase = Find_Storm_Phase(DF_Storm, Start_Time, End_Time)
        for j in range(len(Storm_Phase)):
            Storm_Phase_List.append(Storm_Phase[j])
    Main_DF["Storm Phase"] = Storm_Phase_List
    Main_DF = Main_DF.drop("level_0", axis=1)
    return (Main_DF)

In [35]:
# Function to Find Storm Phase
def Find_Storm_Phase(DF_Storm, Start_Time, End_Time):
    Time = list(DF_Storm["Time(Z)"])
    B = list(DF_Storm["B"])
    VLT = list(DF_Storm["VLT"])
    Storm_Phase = []
    for k in range(len(DF_Storm)):
# If Before ET Start = Tropical
        if Time[k] < Start_Time:
            Storm_Phase.append("Tropical")
# If Between ET Start and ET End = Transitioning
        elif Time[k] < End_Time:
            Storm_Phase.append("Transition")
# If After ET End = Extratropical
        else:
            Storm_Phase.append("Extratropical")
    return (Storm_Phase)

In [36]:
# Apply Function
Control_DF_Final = Storm_Phase_Info(Control_DF_Filtered, Control_ET_Orig)
RCP45_DF_Final = Storm_Phase_Info(RCP45_DF_Filtered, RCP45_ET_Orig)
RCP85_DF_Final = Storm_Phase_Info(RCP85_DF_Filtered, RCP85_ET_Orig)

In [37]:
# Add ET Transition Lat Lon SLP Into ET DataFrame
def ET_Lat_Lon(Main_DF, ET_DF):
    Array = numpy.zeros((7, len(ET_DF)))
    for i in range(len(ET_DF)):
        Code = ET_DF["Code"][i]
        Start_Time = ET_DF["Start Time"][i]
        End_Time = ET_DF["End Time"][i]
        Storm = Main_DF[Main_DF["Code"] == Code]
        Array[0][i] = numpy.min(Storm["SLP(hPa)"])
# Find SLP, Lon, Lat at ET Start and ET End
        Array[1][i] = float(Storm[Storm["Time(Z)"] == Start_Time]["SLP(hPa)"].iloc[0])
        Array[2][i] = float(Storm[Storm["Time(Z)"] == End_Time]["SLP(hPa)"].iloc[0])
        Array[3][i] = float(Storm[Storm["Time(Z)"] == Start_Time]["Lon"].iloc[0])
        Array[4][i] = float(Storm[Storm["Time(Z)"] == Start_Time]["Lat"].iloc[0])
        Array[5][i] = float(Storm[Storm["Time(Z)"] == End_Time]["Lon"].iloc[0])
        Array[6][i] = float(Storm[Storm["Time(Z)"] == End_Time]["Lat"].iloc[0])
    ET_DF["Min SLP"] = Array[0]
    ET_DF["Start SLP"] = Array[1]
    ET_DF["End SLP"] = Array[2]
    ET_DF["Start Lon"] = Array[3]
    ET_DF["Start Lat"] = Array[4]
    ET_DF["End Lon"] = Array[5]
    ET_DF["End Lat"] = Array[6]
    return (ET_DF)

In [38]:
# Function to Add Variables Into ET DataFrame
def ET_Variables(Final_DF, ET_DF):
    ET_DF = ET_Lat_Lon(Final_DF, ET_DF)
    return (ET_DF)

In [39]:
# Apply Function
Control_ET_Final = ET_Variables(Control_DF_Final, Control_ET_Orig)
RCP45_ET_Final = ET_Variables(RCP45_DF_Final, RCP45_ET_Orig)
RCP85_ET_Final = ET_Variables(RCP85_DF_Final, RCP85_ET_Orig)

In [40]:
Control_ET_Final[5:10]

,Code,Name,Path Type,Start Time,End Time,Min SLP,Start SLP,End SLP,Start Lon,Start Lat,End Lon,End Lat
5,TC190105,Lydia,1,1901-09-27 12:00:00,1901-09-29 18:00:00,913.33,953.76,970.00,-53.37,39.19,-42.82,48.53
6,TC190106,Mario,1,1901-09-22 00:00:00,1901-09-22 18:00:00,988.80,993.69,991.98,-78.47,35.78,-71.20,41.68
7,TC190107,Nicole,2,1901-10-14 12:00:00,1901-10-15 00:00:00,974.56,1015.94,1019.52,-22.00,42.00,-26.00,41.25
8,TC190109,Shannon,1,1901-12-14 00:00:00,1901-12-14 06:00:00,942.03,975.20,975.97,-32.75,47.00,-30.25,47.75
9,TC190201,Tony,2,1902-01-27 18:00:00,1902-01-28 00:00:00,1002.71,1012.65,1014.76,-43.50,29.50,-44.50,28.00


In [41]:
Control_Storms_Final[8:13]

,Code,Name,Path Type,Start Time,End Time
8,TC190104,Kinen,0,1901-09-11 00:00:00,NaT
9,TC190105,Lydia,1,1901-09-27 12:00:00,1901-09-29 18:00:00
10,TC190106,Mario,1,1901-09-22 00:00:00,1901-09-22 18:00:00
11,TC190107,Nicole,2,1901-10-14 12:00:00,1901-10-15 00:00:00
12,TC190108,Phil,-1,1901-10-14 18:00:00,NaT


In [42]:
Control_DF_Final[265:277]

,Code,Name,Lon,Lat,SLP(hPa),Winds(m/s),B,VLT,VUT,Time(Z),Storm Phase
265,TC190105,Lydia,-60.54,28.77,920.25,64.4,-3.11,372.15,446.44,1901-09-23 00:00:00,Tropical
266,TC190105,Lydia,-60.94,28.96,919.96,65.9,-5.57,372.20,460.03,1901-09-23 06:00:00,Tropical
267,TC190105,Lydia,-61.51,29.14,917.78,65.1,-4.46,370.60,471.26,1901-09-23 12:00:00,Tropical
268,TC190105,Lydia,-61.38,29.60,916.02,67.2,-3.70,367.19,472.79,1901-09-23 18:00:00,Tropical
269,TC190105,Lydia,-60.97,29.81,913.33,69.7,-2.31,364.90,468.96,1901-09-24 00:00:00,Tropical
270,TC190105,Lydia,-60.56,30.26,918.92,60.6,-2.41,364.34,463.49,1901-09-24 06:00:00,Tropical
271,TC190105,Lydia,-59.94,30.82,923.42,63.6,3.00,358.99,457.63,1901-09-24 12:00:00,Tropical
272,TC190105,Lydia,-59.85,31.73,921.07,57.2,5.25,351.29,451.37,1901-09-24 18:00:00,Tropical
273,TC190105,Lydia,-59.54,32.75,925.31,57.5,7.92,349.72,434.34,1901-09-25 00:00:00,Tropical
274,TC190105,Lydia,-59.99,33.40,929.70,50.7,11.02,335.03,417.31,1901-09-25 06:00:00,Tropical


In [43]:
Control_Codes_Final[5:10]

,ABC,Orig Code,New Code,Name
5,A,0089,TC190105,Lydia
6,A,0090,TC190106,Mario
7,A,0095,TC190107,Nicole
8,A,0108,TC190109,Shannon
9,A,0116,TC190201,Tony


In [44]:
# Output DF to csv File
def Output_File(DF, Model, Type):
    File_Name = str(Model+'_'+Type+'_Output_V4.csv')
    Output_Diri = '/glade/u/home/whimkao//ExtraTrack/Output_Files/'
    DF.to_csv(Output_Diri+File_Name)

In [45]:
# Output Control Files
Output_File(Control_DF_Final, "Control", "Data")
Output_File(Control_ET_Final, "Control", "ET")
Output_File(Control_Codes_Final, "Control", "Codes")
Output_File(Control_Storms_Final, "Control", "Storms")

In [46]:
# Output RCP4.5 Files
Output_File(RCP45_DF_Final, "RCP45", "Data")
Output_File(RCP45_ET_Final, "RCP45", "ET")
Output_File(RCP45_Codes_Final, "RCP45", "Codes")
Output_File(RCP45_Storms_Final, "RCP45", "Storms")

In [47]:
# Output RCP8.5 Files
Output_File(RCP85_DF_Final, "RCP85", "Data")
Output_File(RCP85_ET_Final, "RCP85", "ET")
Output_File(RCP85_Codes_Final, "RCP85", "Codes")
Output_File(RCP85_Storms_Final, "RCP85", "Storms")